In [1]:
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)

/home/bruno/Desktop/coarse-discourse-validation


In [2]:
import pandas as pd
import nltk
import numpy as np
import math

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/bruno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df_path = os.path.join(PROJ_ROOT, 'data', 'interim', 'clean_data.pkl')

df = pd.read_pickle(df_path)

df.head()

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...


In [4]:
df_sample = df.iloc[0:1000]

df_sample.head()

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...


# Features Exploration

In here, we'll be working on the features described on the paper.

## Structure

> We calculate several features related to the structure of the comment and its position. One feature is the depth of the comment according to Reddit’s threaded structure, which we collect as both a raw count and normalized by the number of comments in the discussion. We also calculate number of sentences, number of words, and number of characters of both the body and the title of the comment. We computed these values for both the current comment and the parent comment.

### Post depth

In [5]:
df_sample['post_depth'].value_counts()

1.0     412
2.0     210
3.0     129
0.0     107
4.0      73
5.0      31
6.0      15
7.0       8
8.0       6
10.0      5
9.0       4
Name: post_depth, dtype: int64

In [6]:
# df_sample['post_depth_normalized'] = df_sample['post_depth'] / df_sample['comments_in_discussion']

### Number of sentences

We use NLTK.tokenize to get setences tokenized and then counted.

In [7]:
df_sample['body'].iloc[42]

"You won't regret keeping it. As far as longevity goes, it is almost indefinite. You will wear out parts like springs, mags, and maybe even the grips. The slide, frame, and barrel will likely outlive you. Colt makes great guns. How many companies who produced pistols 100 years ago still have their products found on gun shop shelves in working and safe condition?"

In [8]:
nltk.download('punkt')

nltk.tokenize.sent_tokenize(df_sample['body'].iloc[42])

[nltk_data] Downloading package punkt to /home/bruno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


["You won't regret keeping it.",
 'As far as longevity goes, it is almost indefinite.',
 'You will wear out parts like springs, mags, and maybe even the grips.',
 'The slide, frame, and barrel will likely outlive you.',
 'Colt makes great guns.',
 'How many companies who produced pistols 100 years ago still have their products found on gun shop shelves in working and safe condition?']

In [9]:
sent_number = lambda b : len(nltk.tokenize.sent_tokenize(b))

df_sample['n_sentences'] = df_sample['body'].map(sent_number)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5


### Number of words

In [10]:
nltk.tokenize.word_tokenize(df_sample['body'].iloc[42])

['You',
 'wo',
 "n't",
 'regret',
 'keeping',
 'it',
 '.',
 'As',
 'far',
 'as',
 'longevity',
 'goes',
 ',',
 'it',
 'is',
 'almost',
 'indefinite',
 '.',
 'You',
 'will',
 'wear',
 'out',
 'parts',
 'like',
 'springs',
 ',',
 'mags',
 ',',
 'and',
 'maybe',
 'even',
 'the',
 'grips',
 '.',
 'The',
 'slide',
 ',',
 'frame',
 ',',
 'and',
 'barrel',
 'will',
 'likely',
 'outlive',
 'you',
 '.',
 'Colt',
 'makes',
 'great',
 'guns',
 '.',
 'How',
 'many',
 'companies',
 'who',
 'produced',
 'pistols',
 '100',
 'years',
 'ago',
 'still',
 'have',
 'their',
 'products',
 'found',
 'on',
 'gun',
 'shop',
 'shelves',
 'in',
 'working',
 'and',
 'safe',
 'condition',
 '?']

In [11]:
words_number = lambda b : len(nltk.tokenize.word_tokenize(b))

df_sample['n_words'] = df_sample['body'].map(words_number)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115


### Number of characters

Here we create a tokenizer from a RegEx to remove punctuaions and whitespaces, so we can count only characters.

In [12]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

words = tokenizer.tokenize(df_sample['body'].iloc[42])

sum(map(len, words))

289

In [13]:
char_number = lambda b : sum(map(len, tokenizer.tokenize(b)))

df_sample['n_char'] = df_sample['body'].map(char_number)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words,n_char
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499,2009
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22,74
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82,339
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26,84
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115,434


### Parents number of sentences, words and characters

In [14]:
df_sample['parent_n_sentences'] = 0
df_sample['parent_n_words'] = 0
df_sample['parent_n_char'] = 0

for i, row in df_sample.iterrows():
    try:
        parent = df_sample[(df_sample['thread_title'] == row['thread_title'])][df_sample['id'] == row['in_reply_to']].iloc[0]
        df_sample.at[i, 'parent_n_sentences'] = parent['n_sentences']
        df_sample.at[i, 'parent_n_words'] = parent['n_words']
        df_sample.at[i, 'parent_n_char'] = parent['n_char']
    except:
        pass
    
df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words,n_char,parent_n_sentences,parent_n_words,parent_n_char
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499,2009,0,0,0
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22,74,29,499,2009
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82,339,2,22,74
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26,84,4,82,339
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115,434,0,0,0


## Author

> We collect features about the author of the comment, including a binary feature for whether the current commenter is also the commenter of the initial post and a binary feature for whether the current commenter is the same as the parent commenter.

In [15]:
def is_thread_author(row):
    if row['author'] == '[deleted]' :
        return False
    else:
        return row['author'] == row['thread_author']

df_sample['is_thread_author'] = df_sample.apply(is_thread_author, axis=1)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,thread_author,thread_title,url,n_sentences,n_words,n_char,parent_n_sentences,parent_n_words,parent_n_char,is_thread_author
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,DTX120,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499,2009,0,0,0,True
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,22,74,29,499,2009,False
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,4,82,339,2,22,74,True
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,DTX120,DTX120: #87 - Nashville,NaN,2,26,84,4,82,339,False
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,Keatonus,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115,434,0,0,0,True


In [16]:
def is_parent_author(row):
    try:
        parent_author = df_sample[df_sample['id']==row['in_reply_to']].iloc[0]['author']
        
        return row['author'] == parent_author and row['author'] != '[deleted]'
    except IndexError:
        return False

df_sample['is_parent_author'] = df_sample.apply(is_parent_author, axis=1)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,...,thread_title,url,n_sentences,n_words,n_char,parent_n_sentences,parent_n_words,parent_n_char,is_thread_author,is_parent_author
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,...,DTX120: #87 - Nashville,https://www.reddit.com/r/100movies365days/comm...,29,499,2009,0,0,0,True,False
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,...,DTX120: #87 - Nashville,NaN,2,22,74,29,499,2009,False,False
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,...,DTX120: #87 - Nashville,NaN,4,82,339,2,22,74,True,False
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,...,DTX120: #87 - Nashville,NaN,2,26,84,4,82,339,False,False
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,...,"Male, 23 years old. Going for 100 sets!",https://www.reddit.com/r/100sets/comments/omv7...,5,115,434,0,0,0,True,False


In [17]:
df_sample['is_parent_author'].value_counts()

False    982
True      18
Name: is_parent_author, dtype: int64

In [18]:
df_sample['is_thread_author'].value_counts()

False    756
True     244
Name: is_thread_author, dtype: int64

## Thread

> We calculate features that are the same across all comments in the thread. One feature is the total number of comments in the discussion. Another is the number of unique branches in the discussion tree. We also record whether the discussion originated as a self-post or a link-post. Finally, we collect the average length of all the branches or threads of discussion in the discussion tree.

Most of these may be included in the pre-processing step.

## Community

>We have a feature naming the subreddit that the thread came from, as some subreddits have a greater porportion of some types of discourse and not others.

In [19]:
df_sample['subreddit'] = df_sample['subreddit'].astype('category')

df_sample['subreddit']

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0        100movies365days
1        100movies365days
2        100movies365days
3        100movies365days
4                 100sets
5                 100sets
7                 100sets
8                 100sets
9                 100sets
10                100sets
11           1200isplenty
12           1200isplenty
13           1200isplenty
14           1200isplenty
15           1200isplenty
16           1200isplenty
17           1200isplenty
18           1200isplenty
19                   1911
20                   1911
21                   1911
22                   1911
23                   1911
24                   1911
25                   1911
26                   1911
27                   1911
28                   1911
29                   1911
30                   1911
              ...        
980         adviceanimals
981         adviceanimals
982         adviceanimals
983         adviceanimals
984         adviceanimals
985         adviceanimals
986         adviceanimals
987         

## Content + Punctuation

>We collect unigrams, bigrams, and trigrams from the text of the comment. If the comment has a title, in the case of the initial post, then the n-grams of the title are counted separately from the n-grams of the body. We use a word tokenizer that tokenizes punctuation instead of stripping it so that we count potentially important punctuation like question marks or exclamation points. We use TF-IDF weighting and set a minimum document frequency of 50 comments.

https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
https://www.nltk.org/book/ch06.html - 2.2

### Using nltk ngram functions and Counter object

unigrams_frequency = Counter()
bigrams_frequency = Counter()
trigrams_frequency = Counter()

for body in df_sample['body']:
    tokens = nltk.word_tokenize(body)
    unigrams = tokens
    bigrams = nltk.bigrams(tokens)
    trigrams = nltk.trigrams(tokens)
    
    unigrams_frequency += Counter(unigrams)
    bigrams_frequency += Counter(bigrams)
    trigrams_frequency += Counter(trigrams)

unigrams_frequency.most_common(10)

bigrams_frequency.most_common(10)

trigrams_frequency.most_common(15)

def unigram_vectorizer(body_text):
    features = {}
    for unigram in list(unigrams_frequency.keys()):
        features[unigram] = False
    for word in nltk.word_tokenize(body_text):
        features[word] = True
    return features

def bigram_vectorizer(body_text):
    features = {}
    for bigram in list(bigrams_frequency.keys()):
        features[bigram] = False
    for bigram in nltk.bigrams(nltk.word_tokenize(body_text)):
        features[bigram] = True
    return features

def trigram_vectorizer(body_text):
    features = {}
    for trigram in list(trigrams_frequency.keys()):
        features[trigram] = False
    for trigram in nltk.trigrams(nltk.word_tokenize(body_text)):
        features[trigram] = True
    return features

len(trigrams_frequency.keys())

feature = trigram_vectorizer(df_sample.iloc[42].body)

len(feature)

We get the feature vectors but to get the weighting correctly we would have some more work to do.

### Using sklearn methods

tokens_vector = df_sample['body'].apply(nltk.word_tokenize)

tokens_vector.head()

unigram_vectorizer = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'), lowercase=False, tokenizer=nltk.word_tokenize, analyzer='word', ngram_range=(1, 1), min_df=495)

X = unigram_vectorizer.fit_transform(df_sample['body'])

unigram_vectorizer.get_feature_names()

bigram_vectorizer = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'), lowercase=False, tokenizer=nltk.word_tokenize, analyzer='word', ngram_range=(2, 2), min_df=83)

X = bigram_vectorizer.fit_transform(df_sample['body'])

bigram_vectorizer.get_feature_names()

trigram_vectorizer = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'), lowercase=False, tokenizer=nltk.word_tokenize, analyzer='word', ngram_range=(3, 3), min_df=20)

X = trigram_vectorizer.fit_transform(df_sample['body'])

trigram_vectorizer.get_feature_names()

Because of inconsistency with the results from NLTK ngram functions, an approach based on https://gist.github.com/himzzz/4105717 will be used.

### TF-IDF implementation

nltk_body = df_sample['body'].apply(lambda body: (nltk.word_tokenize(body)))

In [20]:
def tf(word, doc):
    return doc.count(word) / float(len(doc))

def docs_containing(word, docs):
    f = lambda doc: doc.count(word) > 0
    return list(filter(f, docs))

def idf(word, docs):
    return math.log( len(docs) / float(len(docs_containing(word, docs))) )

unigrams_frequency = Counter()
bigrams_frequency = Counter()
trigrams_frequency = Counter()
TFIDF = []

for raw_text in df_sample['body']:
    tokens = nltk.word_tokenize(raw_text)
    text = nltk.Text(tokens)
    
    unigrams = tokens
    bigrams = list(nltk.bigrams(tokens))
    trigrams = list(nltk.trigrams(tokens))

    unigrams_frequency += Counter(unigrams)
    bigrams_frequency += Counter(nltk.bigrams(tokens))
    trigrams_frequency += Counter(nltk.trigrams(tokens))

for raw_text in df_sample['body']:
    tokens = nltk.word_tokenize(raw_text)
    text = nltk.Text(tokens)
    
    unigrams = tokens
    bigrams = list(nltk.bigrams(tokens))
    trigrams = list(nltk.trigrams(tokens))

    ngrams = []
    ngrams.extend(unigrams)
    ngrams.extend(bigrams)
    ngrams.extend(trigrams)
    
    body = {}
    for token in ngrams:
        if len(token) == 1:
            body[token] = tf(token, unigrams) * idf(token, list(unigrams_frequency.keys()))
        elif len(token) == 2:
            body[token] = tf(token, bigrams) * idf(token, list(bigrams_frequency.keys()))
        elif len(token) == 3:
            body[token] = tf(token, trigrams) * idf(token, list(trigrams_frequency.keys()))
    
    TFIDF.append(body)

df_sample['TFIDF'] = pd.Series(TFIDF)

Too much headache, let's try the pandas way.

#### Pandas approach

In [21]:
df_sample['tokens'] = df_sample['body'].apply(lambda body: nltk.word_tokenize(body))
df_sample['bigrams'] = df_sample['tokens'].apply(lambda tokens : list(nltk.bigrams(tokens)))
df_sample['trigrams'] = df_sample['tokens'].apply(lambda tokens : list(nltk.trigrams(tokens)))

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [22]:
unigrams_frequency = df_sample['tokens'].apply(Counter).sum()
bigrams_frequency = df_sample['bigrams'].apply(Counter).sum()
trigrams_frequency = df_sample['trigrams'].apply(Counter).sum()

In [23]:
df_sample['TF-IDF'] = [dict() for _ in range(len(df_sample))]

for i, row in df_sample.iterrows():
    # TODO: Change code to make feature vector already as columns from here
    for token in row['tokens']:
        row['TF-IDF'][token] = tf(token, row['tokens']) * idf(token, df_sample['tokens'])
    
    try:
        for token in row['bigrams']:
            row['TF-IDF'][token] = tf(token, row['bigrams']) * idf(token, df_sample['bigrams'])
    except ZeroDivisionError:
        print(token)
        raise
    
    for token in row['trigrams']:
        row['TF-IDF'][token] = tf(token, row['trigrams']) * idf(token, df_sample['trigrams'])
    
    # df_sample.at[i, 'TF-IDF'] = tfidf

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
df_sample.iloc[10]['TF-IDF']

{'I': 0.05134503209512621,
 'love': 0.06876960046169048,
 'cheese': 0.10466924306872122,
 'cake': 0.06223202954037961,
 '!': 0.030599443714247138,
 'both': 0.031896031072033,
 'making': 0.03725375276344465,
 'and': 0.026725984451594336,
 'eating': 0.049742891151912126,
 'it': 0.03105009473256369,
 ',': 0.04734625273672136,
 'so': 0.018380367824563554,
 "'m": 0.020654367226494393,
 'sad': 0.049742891151912126,
 'to': 0.007074436661871631,
 'see': 0.08321929412143357,
 'that': 0.049242945363016606,
 'they': 0.02231476108233174,
 'usually': 0.04062936942507897,
 'have': 0.014589984218807224,
 'the': 0.006802005263766061,
 'most': 0.031009183569210907,
 'crazy': 0.04609005234012687,
 'calorie': 0.06223202954037961,
 'counts': 0.055987460346145865,
 'do': 0.015398722952399201,
 "n't": 0.02761219587924274,
 'many': 0.07238528866821571,
 'low': 0.039124377671356764,
 'cal': 0.055987460346145865,
 'recipes': 0.06223202954037961,
 'for': 0.033737109816824755,
 'them': 0.024764614374977165,
 'es

Now that we got TF-IDF weights, let's build the feature vector!

In [25]:
def weighted_vectorizer(token_list, token_dict):
    feature = []
    
    for token in token_list:
        try:
            feature.append(token_dict[token])
        except KeyError:
            feature.append(0)
    
    return feature


df_sample['unigrams_feature'] = None
df_sample['bigrams_feature'] = None
df_sample['trigrams_feature'] = None

top_unigrams = [tk for tk, freq in unigrams_frequency.most_common() if freq >= 50]
top_bigrams = [tk for tk, freq in bigrams_frequency.most_common() if freq >= 50]
top_trigrams = [tk for tk, freq in trigrams_frequency.most_common() if freq >= 50]

for i, row in df_sample.iterrows():
    df_sample.at[i, 'unigrams_feature'] = weighted_vectorizer(top_unigrams, row['TF-IDF'])
    df_sample.at[i, 'bigrams_feature'] = weighted_vectorizer(top_bigrams, row['TF-IDF'])
    df_sample.at[i, 'trigrams_feature'] = weighted_vectorizer(top_trigrams, row['TF-IDF'])

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [26]:
df_sample.drop(columns=['tokens', 'bigrams', 'trigrams', 'TF-IDF'], inplace=True)

df_sample.head()

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,annotations,author,body,id,in_reply_to,is_self_post,majority_link,majority_type,post_depth,subreddit,...,n_words,n_char,parent_n_sentences,parent_n_words,parent_n_char,is_thread_author,is_parent_author,unigrams_feature,bigrams_feature,trigrams_feature
0,"[{'link_to_post': 'none', 'main_type': 'announ...",DTX120,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,NaN,True,none,announcement,0.0,100movies365days,...,499,2009,0,0,0,True,False,"[0.02495455514573362, 0.019559302232776677, 0....","[0.017105909557282546, 0.009287692485331513, 0...","[0, 0, 0]"
1,"[{'link_to_post': 't3_1bx6qw', 'main_type': 'a...",mcgrewf10,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,t3_1bx6qw,True,t3_1bx6qw,elaboration,1.0,100movies365days,...,22,74,29,499,2009,False,False,"[0.04042962018740609, 0, 0.07401686444881832, ...","[0.08113088532882579, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0]"
2,"[{'link_to_post': 't1_c9b2nyd', 'main_type': '...",DTX120,You strike me as the type who would appreciate...,t1_c9b30i1,t1_c9b2nyd,True,t1_c9b2nyd,elaboration,2.0,100movies365days,...,82,339,2,22,74,True,False,"[0.021693942539583757, 0.01831161691211175, 0....","[0.02103393323339928, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0]"
3,"[{'link_to_post': 't1_c9b30i1', 'main_type': '...",mcgrewf10,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,t1_c9b30i1,True,t1_c9b30i1,elaboration,3.0,100movies365days,...,26,84,4,82,339,False,False,"[0.03420967862011285, 0.08662803385345173, 0.0...","[0, 0, 0, 0, 0.09676475636999989, 0, 0, 0, 0, ...","[0, 0, 0]"
4,"[{'link_to_post': 'none', 'main_type': 'announ...",Keatonus,"Alright guys, little background about myself. ...",t3_omv7p,NaN,True,none,announcement,0.0,100sets,...,115,434,0,0,0,True,False,"[0.019335905307020302, 0.0522279160623709, 0.0...","[0.01494516308688896, 0.020286275691645147, 0....","[0, 0, 0]"


In [30]:
titles_corpus = pd.Series(list(set(df_sample['thread_title'])))

df_sample['title_tokens'] = df_sample['thread_title'].apply(lambda title: nltk.word_tokenize(title))
df_sample['title_bigrams'] = df_sample['title_tokens'].apply(lambda tokens : list(nltk.bigrams(tokens)))
df_sample['title_trigrams'] = df_sample['title_tokens'].apply(lambda tokens : list(nltk.trigrams(tokens)))

/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/bruno/anaconda3/envs/coarse-discourse-validation/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithC

In [42]:
# Get unique values, excluding comments without title
titles_tokens = pd.Series(list(set(df_sample['thread_title']))).apply(lambda title: nltk.word_tokenize(title))
titles_bigrams = titles_tokens.apply(lambda tokens : list(nltk.bigrams(tokens)))
titles_trigrams = titles_tokens.apply(lambda tokens : list(nltk.trigrams(tokens)))

unigrams_frequency = titles_tokens.apply(Counter).sum()
bigrams_frequency = titles_bigrams.apply(Counter).sum()
trigrams_frequency = titles_trigrams.apply(Counter).sum()

In [46]:
unigrams_frequency.most_common(10)

[('?', 45),
 ('.', 31),
 ('I', 29),
 ('to', 27),
 ('the', 20),
 ('a', 18),
 ('for', 15),
 ("'s", 11),
 (',', 11),
 ('!', 8)]

Since there are no unigrams with a frequency higher than that defined as the lower limit by the paper, we can assume that the title ngrams will be counted with the body. Since there are few titles (less impact) and we are not sure about how to take its influence into account, this **won't be implemented for now**.